In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

# Define the stock universe
us_stock_universe = {
    "Technology": ["AAPL", "MSFT", "GOOGL", "NVDA", "META"],
    "Banking & Financials": ["JPM", "BAC", "WFC", "C", "GS"],
    "Consumer Goods": ["PG", "KO", "PEP", "MO", "MNST"],
    "Pharma & Healthcare": ["JNJ", "PFE", "MRNA", "LLY", "UNH"],
    "Energy": ["XOM", "CVX", "COP", "PSX", "SLB"],
    "Automobile": ["TSLA", "F", "GM", "RIVN", "LCID"],
    "Retail & E-commerce": ["AMZN", "WMT", "COST", "TGT", "HD"],
    "Aerospace & Defense": ["BA", "LMT", "NOC", "GD", "RTX"],
    "Telecom & Media": ["VZ", "T", "DIS", "NFLX", "CMCSA"],
    "Semiconductors": ["AMD", "TSM", "INTC", "AVGO", "QCOM"]
}

# Flatten stock universe to get all tickers
stocks = [ticker for sector in us_stock_universe.values() for ticker in sector]

# Define benchmark (S&P 500)
benchmark_ticker = "^GSPC"

# Define period for historical data
period = "5y"

# Approximate risk-free rate (4% annually)
risk_free_rate = 0.04

# Fetch benchmark data
benchmark_data = yf.Ticker(benchmark_ticker).history(period=period)["Close"]
benchmark_return = (benchmark_data.iloc[-1] / benchmark_data.iloc[0]) - 1  # Total return over the period

# Store Alpha values
alpha_data = []

for stock in stocks:
    try:
        stock_data = yf.Ticker(stock).history(period=period)["Close"]
        stock_return = (stock_data.iloc[-1] / stock_data.iloc[0]) - 1  # Total return over the period

        # Fetch Beta directly from Yahoo Finance
        beta = yf.Ticker(stock).info.get("beta", None)

        if beta is not None:
            # Calculate Alpha
            alpha = (stock_return - risk_free_rate) - (beta * (benchmark_return - risk_free_rate))

            # Store data
            alpha_data.append({"Ticker": stock, "Alpha (Annualized)": alpha / 5})  # Convert to annualized Alpha

    except Exception as e:
        print(f"Error fetching Alpha for {stock}: {e}")

# Save Alpha values
alpha_df = pd.DataFrame(alpha_data)
alpha_df.to_csv("alpha.csv", index=False)

print("✅ Alpha values saved to alpha.csv")
